In [1]:
import pandas as pd
from conllu import parse

from finding_my_triplets import create_path_and_file_name, read_preprocess_data, create_text_file

In [2]:
name = 'izbavitsya-21'

In [3]:
names = create_path_and_file_name(name)

In [4]:
df = read_preprocess_data(names)

In [5]:
df.head()

,Left context,Center,Punct,Right context,Created,Full context
0,Хочу спросить совета ― можно ли,избавиться,,от краски на брюках самостоятельно или,1970-2000,Хочу спросить совета ― можно ли избавиться от ...
1,сразу ― и хладнокровно от нее,избавился,",","сказать точнее ― оприходовал, загнал в пробирку",1991-2000,Ты это понял сразу ― и хладнокровно от нее изб...
2,"нужно поскорей, словно от морока,",избавиться,,"от этих шариков, скрипящих и скрипящих",1991-2000,"Всем существом хотелось повторить, а значит, б..."
3,"прежде надо бы унять тахикардию,",избавиться,,от онемения едва ль не в,1991-2000,"Навек уйти из павильона, так; но прежде надо б..."
4,"всякой надобности, лишь бы скорей",избавиться,,"от запаха валокордина, вновь вышел на",1991-2000,"Без всякой надобности, лишь бы скорей избавить..."


In [6]:
create_text_file(df, names)

## Ввести названия файлов для парсера

In [7]:
print(names[1], names[2], sep = ' > ')

sents_izbavitsya-21.txt > parsed_sents_izbavitsya-21.conllu


In [8]:
!udpipe --tokenizer=presegmented --tag --parse\
    ../models/russian-syntagrus-ud-2.0-170801.udpipe sents_izbavitsya-21.txt > parsed_sents_izbavitsya-21.conllu

Loading UDPipe model: done.


## Распределение по годам

In [9]:
from finding_my_triplets import read_conllu, get_all_triples, get_standart_date, get_indexes, \
count_necessary_triplets, show_preview_for_sent

In [10]:
# name = 'kasatsya-19-2'

object_form = {'Case': 'Gen'}
verb_lemmas = ['избавиться', 'избавляться']
prep_in_var_of_constr = None
prep_in_constr = ['от']

necessary_time_period = (2000, 2049)

In [11]:
names = create_path_and_file_name(name)
df = read_preprocess_data(names)

In [12]:
sentences = read_conllu(names[2])
triplets = get_all_triples(sentences, verb_lemmas, object_form, prep_in_var_of_constr, prep_in_constr)
ids = get_indexes(triplets) # индексы предложений, которые подходят и для которых узнаём дату

In [15]:
# triplets

In [16]:
distrib_by_time_periods = {
    (1700, 1749): [],
    (1750, 1799): [],
    (1800, 1849): [],
    (1850, 1899): [],
    (1900, 1949): [],
    (1950, 1999): [],
    (2000, 2049): []
}

for ind in ids:
    date = df.iloc[ind].Created # получение по индексу даты создания из DataFrame
    date = get_standart_date(date)
    date = int(date)
    for start_year, end_year in distrib_by_time_periods:
        if date >= start_year and date <= end_year:
            distrib_by_time_periods[(start_year, end_year)].append(ind)

In [17]:
sent_ids = distrib_by_time_periods[necessary_time_period]
necessary_triplets = []
for triplet in triplets:
    if 'object' in triplet and triplet['id'] in sent_ids:
        necessary_triplets.append(triplet)

In [18]:
count_necessary_triplets(necessary_triplets, sentences)

id 480 избавиться полностью от культуральности , как основы


1

In [19]:
sentences[0]

TokenList<Хочу, спросить, совета, ―, можно, ли, избавиться, от, краски, на, брюках, самостоятельно, или, лучше, обратиться, к, специалистам, ?>

In [20]:
len(triplets)

571